In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df= pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y= df['loan_status']

# Separate the X variable, the features
X= df.drop(columns='loan_status')

In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values (96% of data is "healthy loan status")
y.value_counts(normalize=True)

0    0.967757
1    0.032243
Name: loan_status, dtype: float64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train,X_test,y_train,y_test= train_test_split(X,y,stratify=y,random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lrm= LogisticRegression(random_state=1)

# Fit the model using training data
lrm_d= lrm.fit(X_train,y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
testing_p= lrm.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
accu_score= balanced_accuracy_score(y_test,testing_p) 
print(f"The balancd accuracy score of the LRM model is {round(accu_score*100,2)}%")

The balancd accuracy score of the LRM model is 94.43%


In [11]:
# Testing the normal accuracy score
from sklearn.metrics import accuracy_score
ac= accuracy_score(y_test,testing_p)
print(str(round(ac*100,2))+ " %")

99.24 %


In [12]:
# Generate a confusion matrix for the model
index= ["Actual Healthy", "Actual High Risk"]
columns= ["Predicted Healthy","Predicted High risk"]

c_m= confusion_matrix(y_test,testing_p)
pd.DataFrame(c_m,index=index,columns=columns)

,Predicted Healthy,Predicted High risk
Actual Healthy,18679,80
Actual High Risk,67,558


In [13]:
# Print the classification report for the model
t_n= ["Healthy","High Risk"]
print(classification_report(y_test,testing_p,target_names=t_n))

              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00     18759
   High Risk       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With the classification report that provided the scores: precision, accuracy and recall. Along with the, balanced and "normal accu_score", we see that the model yields 95-99% accuracy of predicting the loan status correctly. However, with a sub 90% precision and recall for high risk score (Loan_status of 1). I would want to increase the data within this dataset, with the purpose of creating a more balanced dataset.

With the ultimate assumption that the model balanced accuracy also increases above a 98% threshold and both precision and recall scores (f1-score) to be above 90%!

---

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
ros= RandomOverSampler(sampling_strategy=.2,random_state=1)


# Fit the original training data to the random_oversampler model
X_train_re, y_train_re= ros.fit_resample(X_train,y_train)

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [15]:
# Count the distinct values of the resampled labels data
pd.Series(y_train_re).value_counts(normalize=True)

0    0.833338
1    0.166662
Name: loan_status, dtype: float64

In [16]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr= LogisticRegression()

# Fit the model using the resampled training data
lr.fit(X_train_re,y_train_re)

# Make a prediction using the testing data
re_p= lr.predict(X_test)

In [17]:
# Print the balanced_accuracy score of the model 
accu_score_2= balanced_accuracy_score(y_test,re_p) 
print(f"The balancd accuracy score of the LRM model is {round(accu_score_2*100,2)}%")

The balancd accuracy score of the LRM model is 98.88%


In [18]:
ac_2= accuracy_score(y_test,re_p)
print(str(round(ac*100,2))+ " %")

99.24 %


In [19]:
# Generate a confusion matrix for the model
index= ["Actual Healthy", "Actual High Risk"]
columns= ["Predicted Healthy","Predicted High risk"]

c_m_2= confusion_matrix(y_test,re_p)
pd.DataFrame(c_m_2,index=index,columns=columns)

,Predicted Healthy,Predicted High risk
Actual Healthy,18669,90
Actual High Risk,11,614


In [20]:
# Print the classification report for the model
t_n= ["Healthy","High Risk"]
print(classification_report(y_test,re_p,target_names=t_n))

              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00     18759
   High Risk       0.87      0.98      0.92       625

    accuracy                           0.99     19384
   macro avg       0.94      0.99      0.96     19384
weighted avg       1.00      0.99      0.99     19384



**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With the introduction of the Oversampled data. All the relative scores improve drastically:
balanced_accuracy_score (new 99%), recall of the high risk (98%). While the balance of recall score improves, thus improving the f1-score.
